# Event selection

In [ ]:
# Lets first import all we need
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from functools import partial
from na62 import prepare, hlf, extract, constants, stats, histo

## Data processing: from raw data to high-level objects
The NA62 data, after being acquired, are first reconstructed. This means we take our raw (binary) data and we extract for each event the signals recorded for each detector individually. For each detectors, these signals are reconstructed as hits containing information like time, position, and geometrical channel ID (i.e. related to physical position rather than electronics position). Hits are further grouped together to reconstruct and form candidates: candidates are more complex objects generally meant to relate to a single particle interacting with the detector. These candidates can be referred to with other names for certain detectors: LKr -> cluster, Spectrometer -> track, GTK -> beam track.

At the analysis level, candidates from multiple detectors can be associated together in space (geometrically) and in time to form higher level objects:
 - beam kaon: GTK track associated to a KTAG candidate
 - downstream track: spectrometer track associated to eventual candidates from CHOD, NewCHOD, MUV1-3, LKr, RICH
 - vertex: association between GTK and spectrometer tracks, or associated between spectrometer tracks only, or association between LKr clusters (neutral vertex)

## Pile-up
As you may have guessed, the data presented to you here consist of events in the form of association of several of these high-level objects. However to reach that point, there has been already a sizable amount of selection performed at analysis level to make these associations. In fact one single event will usually consist in many more of those high-level objects coming from pile-up. Pile-up refers to valid events (either a beam kaon decay, a beam pion decay, a single beam muon) which are *not* (with caveat) related to the event that generated the trigger. In order to make sure we have **all** the information relating to that specific events, we are requesting signals from detector that are spread within ~100 ns of the trigger time. This means we have a lot of pile-up in each event, which is spread in time randomly within the event. But by applying timing cuts at the analysis level, we can easily discard a very large fraction of those pile-up events. There is still a small fraction of events remaining which happen to be at the same time as the triggered event. As it is not possible to distinguish it from the triggered event, additional selection criterion must be applied to ensure it is a valid kaon decay. As the pile-up event is at the same time as the kaon decay, we usually have several possibilities to combine the high-level objects. For instance, if you have one track and 3 isolated clusters and looking for a K2pi decay, you have three association possibilities for the clusters: combine clusters 1 and 2 to form a pi0, combines cluster 2 and 3, or combines clusters 1 and 3. This is called the combinatorics and you can apply conditions on each of all the possible options. If you manage to find a single option that satisfies all your requirements, this could be the correct one and you go ahead forming an event with this selection of high-level object. If not you usually reject the event.

This process of eliminating the pile-up has already been done for you in the data that are available here, and this is the reason why each of your event has only one to three tracks and zero to two clusters. There remains one category of pile-up contaminating your sample (the caveat mentionned above): events that are **not** a kaon decay, but a combination of independent particles that look like a kaon decay. These required additional selection criterion, some of those have already been applied and some that we can apply ourselves in these data.

## Pre-selection
As mentionned above, a comprehensive pre-selection has already been applied to the data available here. You will find below the details on the criterion that have been used for each of the selected 'event_type'.

### K3pi selection
 - Control trigger
 - Exactly one three-track vertex within 6 ns of the trigger time
 - Each track must be in the acceptance of NewCHOD and 4 spectrometer chambers
 - Tracks must be further than 10 cm away from each other at Straw1
 - $q_\text{vtx} = +1$
 - Vertex between 104 m and 180 m and $\chi^2<25$
 - Reconstructed total momentum $p_\text{tot}$ must be within 3 GeV of 75 GeV
 - Reconstructed three-pion invariant mass $m_{3\pi}$ must between 490 MeV and 497 MeV

### Kmu2 selection
 - Control trigger
 - Less than 10 tracks
 - Exactly one good track:
     - Must have CHOD or NewCHOD associated signal
     - Within 10 ns from the trigger time
     - Signal in 4 spectrometer chambers
     - Track $\chi^2<20$
     - Not forming a vertex with any other track. Vertex if:
         - cda < 50 mm
         - z vertex between 60 m and 200 m
 - $q=+1$
 - Track momentum betwen 5 GeV and 70 GeV
 - Track in geometric acceptance of all 4 spectrometer stations, MUV3
 - Vertex between 120 m and 180 m with CDA<40 mm
 - Closest KTAG candidate in time must be within 2 ns of the track CHOD time, or 5 ns from the track NewCHOD time if CHOD time is not available
 - No in-time activity in the LAV, IRC and SAC
 - Track must have MUV3 signal associated to the track within 1.5 ns of the KTAG time, and within 2 ns of the track CHOD time (5 ns of NewCHOD if CHOD not available)
 - Track $E/p < 0.2$
 - Missing mass squared (muon hypothesis) $m_\text{miss}^2(\mu) < 0.02~\text{GeV}^2$

### K2pi selection
 - control trigger
 - At least 1 track and 3 clusters
 - At least 2 good clusters:
     - Within 20 ns of the trigger time
     - Further than 20 mm from a dead cell
     - Cluster must be electromagnetic
     - At least 1 GeV
     - Not associated to a Spectrometer track
     - Must be isolated
 - Build pairs of clusters as pi0:
   - Within 5 ns of each other
   - Further than 200 mm apart
   - Energy sum between 2 GeV and 75 GeV
 - Exactly 1 good track definition:
     - At least one pi0 within 2 ns of the track
     - $q=+1$
     - Not fake
     - Less than 20 GeV between raw momentum and fitted momentum
     - Momentum between 5 GeV and 70 GeV
     - Vertex wrt. beam axis must be between 105 m and 180 m
     - CDA < 30 mm
     - Inside the geometric acceptance of NewCHOD, all 4 Spectrometer stations, LKr
 - Select closest pi0 in time
 - No MUV3 signal associated to the track
 - Must have LKr cluster associated to the track, and $E/p < 0.9$
 - No in-time activity in the LAV, IRC and SAC
 - Reconstructed kaon mass $m_{\pi\gamma\gamma}$ between 460 MeV and 520 MeV
 - Missing mass squared (pion hypothesis) $m_\text{miss}^2(\pi) < 0.015~\text{GeV}^2$

### Kmu3 selection
 - Control trigger
 - At least one track
 - Exactly one good track:
     - Must have MUV3 association
     - Vertex wrt. beam axis must be between 110 m and 180 m
     - CDA < 25 mm
     - Momentum between 5 GeV and 50 GeV
     - $q = +1$
     - Track $\chi^2 < 20$
     - In acceptance of NewCHOD, 4 spectrometer chambers, LKr and MUV3
 - No other track within 10 ns of the good track
 - Must have exactly two good LKr clusters:
     - Energy > 2 GeV
     - Within 6 ns of the track
     - Further than 150 mm from the track impact point on LKr
 - Neutral vertex within 10 m of charged vertex
 - No in-time activity in the LAV, IRC and SAC
 - Reconstructed total momentum $p_\text{tot}$ must be between 15 GeV and 70 GeV
 - Reconstructed transverse total momentum $p_{\text{tot},T}$ must be between 40 and 250 MeV
 - Missing mass squared (muon hypothesis) $m_\text{miss}^2(\mu) < 0.01~\text{GeV}^2$

### Ke3 selection
 - Control trigger
 - At least one track
 - Exactly one good track:
     - Must not have MUV3 association
     - Vertex wrt. beam axis must be between 110 m and 180 m
     - CDA < 25 mm
     - Momentum between 5 GeV and 50 GeV
     - $q = +1$
     - Track $\chi^2 < 20$
     - In acceptance of NewCHOD, 4 spectrometer chambers, LKr and MUV3
 - No other track within 10 ns of the good track
 - Must have exactly two good LKr clusters:
     - Energy > 2 GeV
     - Within 6 ns of the track
     - Further than 150 mm from the track impact point on LKr
 - Neutral vertex within 10 m of charged vertex
 - No in-time activity in the LAV, IRC and SAC
 - Reconstructed total momentum $p_\text{tot}$ must be between 15 GeV and 70 GeV
 - Reconstructed transverse total momentum $p_{\text{tot},T}$ must be between 40 and 250 MeV
 - Missing mass squared (electron hypothesis) $m_\text{miss}^2(e) < 0.01~\text{GeV}^2$



## Criterions categories
From the selections above we can roughly create some "categories" of criterion:
 - Trigger
 - Basic objects requirements
 - Object selection conditions
     - Timing
     - Track quality
     - Charge
     - Momentum
     - Vertex
     - Geometric acceptance
 - Additional requirements
 - Pile-up veto
 - Reconstructed quantities

This can be useful to roughly compare the selections and identify differences. The table below summarises the selection according to these categories

| Type | Sample | Trigger | Basic requirements | O. timing | O. quality | O. charge | O. momentum | O. vertex | O. Geo. Acc. | Add. Req. | Veto | Reco quantities |
| --:    | --:    | :--:    | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: | :--: |
| 3-track | K3pi   | Control | One 3-track vertex | $|t_\text{vtx} - t_\text{trigger}| < 6~\text{ns}$ | $d_{i,j} > 10~\text{cm}$ | +1 | N/A | $104~m < Z_\text{vtx} < 180~m$ & $\chi^2<25$ | NewCHOD & 4 Straw | N/A | N/A | $|p_\text{tot} - 75~\text{GeV}|<3~\text{GeV}$ & $490~\text{MeV} < m_{3\pi} < 497~\text{MeV}$ |
| 1-track | Kmu2   | Control | < 10 tracks | $|t_\text{track} - t_\text{trigger}| < 10~\text{ns}$ | $\chi^2 < 20$ & No other vertex | +1 | $5~\text{GeV} < p < 70~\text{GeV}$ | $120~m < Z_\text{vtx} < 180~m$ & $\text{CDA} < 40~mm$ | MUV3 & 4 Straw | Signal in 4 Straw and (CHOD or NewCHOD) & KTAG in-time & MUV3 in-time signal & $E/p < 0.2$ | LAV, IRC, SAC | $m_\text{miss}^2(\mu) < 0.02~\text{GeV}^2$ |
| 1-track & 2-clusters | K2pi   | Control | 1 track & 3 clusters | $|t_\text{track}-t_{\pi_0}| < 2~\text{ns}$; Cluster: $|t_\text{cluster} - t_\text{trigger}|<20~\text{ns}$ & $\Delta t_{i,j}<5~\text{ns}$ | Track: Not fake & $\Delta p_\text{raw,fit}<20~\text{GeV}$; Cluster: $\Delta d_\text{dead-cell} > 20~\text{mm}$ & Electromagnetic & Not associated to track & Isolated & $\Delta d_{i,j} > 200~\text{mm}$ | Track: +1 | Track: $5~\text{GeV} < p < 70~\text{GeV}$; Cluster: $E>1~\text{GeV}$ & $2~\text{GeV} < E_1 + E_2 < 75~\text{GeV}$ | $105~m < Z_\text{vtx} < 180~m$ & $\text{CDA} < 30~mm$ | NewCHOD & 4 Straw | Track: no MUV3 &  $E/p<0.9$ | LAV, IRC, SAC | $460~\text{MeV} < m_K < 520~\text{MeV}$ & $m_\text{miss}^2(\pi) < 0.015~\text{GeV}^2$ |
| 1-track & 2-clusters | Kmu3 | Control | N/A | $|t_\text{cluster} - t_\text{track}| < 6~\text{ns}$ | Track: $\chi^2 < 20$ & MUV3 association; Cluster: $\Delta d_\text{cls,track} > 150~\text{mm}$ | Track: +1 | Track: $5~\text{GeV} < p < 50~\text{GeV}$; Cluster: $E>2~\text{GeV}$ | Track: $110~m < Z_\text{ch. vtx} < 180~m$ & $\text{CDA} < 25$; Cluster: $|Z_\text{ch. vtx} - Z_\text{neut. vtx}|<10~m$ | Track: NewCHOD & 4 Straw & LKr & MUV3 | No track within 10 ns | LAV, IRC, SAC | $15~\text{GeV} < p_\text{tot} < 70~\text{GeV}$ & $40~\text{MeV} < p_{\text{tot},T} < 250~\text{MeV}$ & $m_\text{miss}^2(\mu) < 0.01~\text{GeV}^2$ |
| 1-track & 2-clusters | Ke3 | Control | N/A | $|t_\text{cluster} - t_\text{track}| < 6~\text{ns}$ | Track: $\chi^2 < 20$ & No MUV3 association; Cluster: $\Delta d_\text{cls,track} > 150~\text{mm}$ | Track: +1 | Track: $5~\text{GeV} < p < 50~\text{GeV}$; Cluster: $E>2~\text{GeV}$ | Track: $110~m < Z_\text{ch. vtx} < 180~m$ & $\text{CDA} < 25$; Cluster: $|Z_\text{ch. vtx} - Z_\text{neut. vtx}|<10~m$ | Track: NewCHOD & 4 Straw & LKr & MUV3 | No track within 10 ns | LAV, IRC, SAC | $15~\text{GeV} < p_\text{tot} < 70~\text{GeV}$ & $40~\text{MeV} < p_{\text{tot},T} < 250~\text{MeV}$ & $m_\text{miss}^2(e) < 0.01~\text{GeV}^2$ |

Even though all the selections seem to contain a similar set of conditions, the cut values are sometimes differing slightly. As we will eventually compare events type selected with different selections, it is best to harmonize the cut values. This will avoid introducing systematic uncertainties due to different behaviour and different agreement between MC and data in different ranges of the variables on which we introduce conditions.

The conditions where we can see obvious differences that can be harmonized are in the following list. When common criterion are selected, we have to use the most constraining values as we cannot "undo" what was already applied before presenting these data.
 - $Z \text{vtx}$: common range is between 120 m and 180 m
 - LKr distance between track and cluster: common limit of at least 150 mm
 - LKr distance between clusters: common limit of at least 200 mm
 - Cluster Energy: common limit of at least 2 GeV
 - Vertex CDA: common limit of less than 25 mm
 - Distance between charged and neutral vertices: common limit of less than 10 m

## Technical aside 
### Functional cuts
As you have seen, we are working in this project with pandas Dataframes. Among other things the dataframe allow us to easily implement cuts, which you have experienced already in the previous notebooks. The way we have done this so far is an operation done in multiple steps (example selecting a track momentum range):
 - Identify all events satisfying the lower range: `low_cond = df["track1_momentum_mag"]>15000`
 - Identify all events satisfying the upper range: `upper_cond = df["track1_momentum_mag"]<40000`
 - Combine both criterion: `cond = low_cond & upper_cond`
 - Select events satisfying both conditions: `df.loc[cond]`

The first three steps actually creates a boolean array with one entry for each event: True or False. Only the last step actually filter out the events we are not interested in. We often shortened this in a single line:
```python
df.loc[(df["track1_momentum_mag"]>15000) & (df["track1_momentum_mag"]<40000)]
df.loc[(df["track2_momentum_mag"]>15000) & (df["track2_momentum_mag"]<40000)]
df.loc[(df["track3_momentum_mag"]>15000) & (df["track3_momentum_mag"]<40000)]
```

However we find that these lines are a bit verbose with a lot of redundancy: `df` is present three times, `track1_momentum_mag` twice. It also lacks flexibility: if we want to apply this condition on all three tracks, we have to write the line three times, with two small changes. Also if we want to try a different value of pmin and pmax, we may have to change this in many places, risking to forget changing it in one place. This could be improved by writing functions and loops, but the code would loose readability. When you read a selection code, you do not care how you applied a momentum cut to three tracks, only that you did and which cut values were used.

Instead we can take advantage of one functionality provided by pandas: the argument of the `.loc()` function can be a `Callable` (i.e. a function or a lambda) taking a single argument (the dataframe) and returning a boolean array used for the filtering. We can therefore use it like this:
```python
def t1_momentum_cut_15_40(df):
    return (df["track1_momentum_mag"]>15000) & (df["track1_momentum_mag"]<40000)
def t2_momentum_cut_15_40(df):
    return (df["track2_momentum_mag"]>15000) & (df["track2_momentum_mag"]<40000)
def t3_momentum_cut_15_40(df):
    return (df["track3_momentum_mag"]>15000) & (df["track3_momentum_mag"]<40000)

df.loc[t1_momentum_cut_15_40]
df.loc[t2_momentum_cut_15_40]
df.loc[t3_momentum_cut_15_40]
```

This is so far not much of an improvement, except for the case where we use these conditions in multiple places. In that case we have to change only one function per track. We can another improvement by using another python propertie: we can write function that create functions!
```python
def make_momentum_cut(min_p, max_p, which_object = None):
    # We can do some pre-processing
    if which_object is None:
        which_object = ""
    else:
        which_object = f"{which_object}_"

    # We define the function from the example above
    def cut(df):
        # This function also works if we provide directly the series!
        if isinstance(df, pd.DataFrame):
            serie_cut = df[f"{which_object}momentum_mag"]
        else:
            serie_cut = df
        # We implement the case where we don't have a lower or an upper limit
        min_momentum_range = serie_cut > min_p if min_p else True
        max_momentum_range = serie_cut < max_p if max_p else True
        return min_momentum_range & max_momentum_range
    # Return the function we created
    return cut

from functools import partial
# Create a new function that binds the min/max parameters, but not which_object
momentum_cut_15_40 = partial(make_momentum_cut, 15000, 40000)

df.loc[momentum_cut_15_40("track1")]
df.loc[momentum_cut_15_40("track2")]
df.loc[momentum_cut_15_40("track3")]
```

True, the overal code is longer, but the on the other hand the interesting part (that last 4 lines) are now much easier to read (reading the line we understand immediately what condition we apply), less error prone (we have a single place to change if we want to modify the cut everywhere), and more flexible (with the same function we can apply to any track without redefinition, and we can easily define another cut in a single line). The `make_momentum_cut` function is implemented in na62.hlf taking even further advantage of those principles by leveraging a generic *min_max* cut function. It is now very short:
```python
def make_momentum_cut(min_p, max_p, which_object = None) -> Callable:
    return make_min_max_cut(min_p, max_p, which_value="momentum_mag", which_object=which_object)
```

Many "make_*_cut" condition functions have been defined in such a way in the na62.hlf module. Have a look at the documentation to see which are available.

### Data/MC histograms
Because we now have MC samples, what we will want to do is to compare our data with the simulation. This is done by plotting together the data and the MC on a single plot. There are however a few things to take into accout:
 - The data comprise the sum of all channels, therefore the MC contributions should be stacked (plotted on **top** of each other and not besides each other).
 - The MC events available for each channel do not represent the same number of kaons. Three parameters have to be taken into account to be able to normalize them relative to each other:
   - The "norm" parameters retrieved in the file. It is the total number of events that were generated (how many kaon decays). Using these we can take into account their relative size (in terms of generation).
   - The branching fraction of each sample.
   - The selection acceptance/efficiency of each sample. The selection cuts do not have the same effect on each sample (because their kinematic distributions are different). 
 - Once the MC samples are correctly normalized relative to each other, the total integrated MC sample must also be normalized to correspond to the number of data events that we have collected.

**Exercise**: Imagine that you simulate 1000 kmu2 events out of which 500 pass your selection, and 2000 K3pi events out of which 100 pass your selection. How many kaons does that corresponds too for each sample? What should be their relative weights? If we select a data sample corresponding to 10000 events (consisting only of kmu2 and k3pi - your selection is very pure), how many kmu2 and k3pi does that correspond to?

In [ ]:
n_kmu2 = 1000
n_k3pi = 2000
n_kmu2_sel = 500
n_k3pi_sel = 100
ndata_sel = 10000

# First compute to number of kaons that correspond to those simulated events
n_kmu2_kaons = n_kmu2 / constants.kaon_br_map["kmu2"]
n_k3pi_kaons = n_k3pi / constants.kaon_br_map["kmu2"]

print(f"Number of Kaons for kmu2: {n_kmu2_kaons:.2f}")
print(f"Number of Kaons for kmu2: {n_k3pi_kaons:.2f}")
print(f"Relative weight: {n_k3pi_kaons/n_kmu2_kaons:.2f} (1 k3pi kaon is worth {n_k3pi_kaons/n_kmu2_kaons:.0f} kmu2 kaons)")

# Now compute the weights for each selected event
w_k3pi = (n_k3pi_sel/n_k3pi_kaons)
w_kmu2 = (n_kmu2_sel/n_kmu2_kaons)
print(f"Relative selected weight: {w_k3pi/w_kmu2:.2f} (1 selected k3pi kaon is worth {w_k3pi/w_kmu2:.0f} selected kmu2 kaons)")

mc_normalization = w_k3pi + w_kmu2

print(f"Number of Kmu2 in the data: {ndata_sel*w_kmu2/mc_normalization:.0f}")
print(f"Number of K3pi in the data: {ndata_sel*w_k3pi/mc_normalization:.0f}")

There are two ways to normalize your MC histograms to the data.
 1. Make sure that your MC histograms are correctly normalized relative to each other, then scale them all so that their integral is the same as the integral of the data. This method is simpler to apply early as it only requires a data histogram and MC histograms without additional ingredients. However this assumes that the events in all the histograms have been rigorously selected in the exact same way. Else the integral will be identical but some features may look very different.
 2. Implement a normalization selection (see later in the notebook) to determine a kaon flux (how many kaons entered the experiment) and scale everything relative to that number. This has the advantage that this number is valid whatever selection criterion are applied. This will allow us to better compare data and MC and clearly see features that are not conssistent between the two, without having it diluted through the whole histogram.
    
**Example**: To illustrate the two methods above, we are going to generate a dataset according to a complex distribution (representing data), and another dataset with a slightly different distribution (representing MC). We are then going to produce histograms according to both methods discussed above.

In [ ]:
# First define our test distributions: 
#  1. two gaussian with completely different parameters on top of a uniform distribution. The amplitude of the second gaussian can be modified from parameter.
#  2. One simple gaussian
# These distribution will correspond to two "channel" featuring different behaviour.
from lmfit.models import GaussianModel, LinearModel
def test_distribution1(x, amp2=100):
    # Create the model
    distrib_model = GaussianModel(prefix="g1_") + GaussianModel(prefix="g2_") + LinearModel()

    # Set the parameters
    pars = distrib_model.make_params()
    pars["g1_center"].value = 2
    pars["g1_sigma"].value = 0.5
    pars["g1_amplitude"].value = 60
    pars["g2_center"].value = 5
    pars["g2_amplitude"].value = amp2
    pars["intercept"].value = 30
    pars["slope"].value = -3

    # Return the model evaluated on the provided x values
    return distrib_model.eval(x=x, params=pars)

def test_distribution2(x):
    # Create the model
    distrib_model = GaussianModel(prefix="g1_")

    # Set the parameters
    pars = distrib_model.make_params()
    pars["g1_center"].value = 2
    pars["g1_sigma"].value = 0.5
    pars["g1_amplitude"].value = 20

    # Return the model evaluated on the provided x values
    return distrib_model.eval(x=x, params=pars)
    
# We will then randomly pick a dataset corresponding to our distribution. We can do this 
# using the accept-reject method
def accept_reject(distrib, range, max, n_samples, plot_it=False):
    # Generate x and y uniformly distributed in a 2D box
    x = np.random.uniform(range[0], range[1], n_samples)
    y = np.random.uniform(0, max, n_samples)
    # Compute for each x the expected y value according to the input distribution
    y_test = distrib(x)

    # Accept only the values which are below the curve
    accept = np.where(y<=y_test)
    reject = np.where(y>y_test)

    # We can plot to illustrate
    if plot_it:
        plt.scatter(x[reject], y[reject], marker=".", label="Rejected")
        plt.scatter(x[accept], y[accept], marker=".", label="Accepted")
        plt.scatter(x, y_test, marker="x", label="Exact distribution")
        plt.legend(loc="lower left")
        plt.xlabel("x")
        plt.ylabel("y")
    return x[accept]

distrib_max = 100 # For accept-reject method
# We are going to generate data corresponding to a certain flux, and a number of MC that we fix (to have something equivalent to our physics problem with data and MC).
# We are also going to simulate the fact that each MC sample has a different branching ratio (0.75 and 0.25)
flux = 100000
n_simu = 200000
n_simu2 = 150000
br1 = 0.75
br2 = 0.25

x = np.arange(0,10,0.1)
ratio = 0.7528 # Don't worry about this number. It's a trick to keep the BR of the distribution1 the same after changing the amplitude of the second gaussian

# Generate the data
data = accept_reject(lambda x: test_distribution1(x) + test_distribution2(x), (0,10), distrib_max, flux, plot_it=True)
# Generate the MC twice using the first distribution: once with the same distribution as the data, once changing the amplitude of the second Gaussian 
mc_same = accept_reject(test_distribution1, (0,10), distrib_max*br1, n_simu)
plt.figure()
mc = accept_reject(partial(test_distribution1, amp2=50), (0,10), distrib_max*ratio, n_simu)
# And once for the second distribution
mc2 = accept_reject(test_distribution2, (0,10), distrib_max*br2, n_simu2)
# Then we do a third sample from the one with the same distribution as the data, but we remove all the events 
# with x<1 (this would correspond to an additional cut in a data selection)
mc_same_with_cut = mc_same[mc_same>1]
mc2_with_cut = mc2[mc2>1]

First please notice the nice illustration of the "Accept-Reject" sampling method, where we randomly generate points in a 2D space and reject all those that are above the theoretical curve that we defined.

Now below let's histogram our data. Three histograms according to the first histograming method

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(21,5))
weights = [0.75/n_simu, 0.25/n_simu2]
histo.hist_data(data, bins=100, range=(0,10), ax=ax[0])
histo.stack_mc_scale([mc_same, mc2], bins=100, range=(0,10), ndata=len(data), weights=weights, ax=ax[0])
ax[0].set_title("MC generated according to same distribution as data")
ax[0].set_xlabel("x")

histo.hist_data(data, bins=100, range=(0,10), ax=ax[1])
histo.stack_mc_scale([mc_same_with_cut, mc2_with_cut], bins=100, range=(0,10), ndata=len(data), weights=weights, ax=ax[1])
ax[1].set_title("MC generated according to same distribution as data\n, wihth cut x>1")
ax[1].set_xlabel("x")

histo.hist_data(data, bins=100, range=(0,10), ax=ax[2])
histo.stack_mc_scale([mc, mc2], bins=100, range=(0,10), ndata=len(data), weights=weights, ax=ax[2])
ax[2].set_title("MC generated according to distribution where\n the amplitude of the second sigma is different")
ax[2].set_xlabel("x")

On these plots, we can clearly see the disadvantave of this normalization technique:
 - Everything looks fine on the first plot, which has been carefully engineered to have MC samples corresponding to the data
 - On the second plot, we added an additional cut to the MC and as a result, the entirety of the MC histograms are shifted upwards.
 - The the last plot, the largest MC contribution is actually slightly different from the data (the second gaussian is a bit smaller) and as a result the whole plot looks bad, including the first Gaussian which is actually correct in MC.

On the other hand these plots have been normalized easily, with the only required input being the BR and the number of generated MC samples. Those values are generally easy to use as they are provided externally (you know the BR of your kaon decay channels, and you know how many events you simulated).

Now for the second normalization technique.

In [ ]:
# The `stack_mc_flux` function uses internally the `constants.kaon_br_map` for the BR of each sample. So let's introduce fake channels with the BR that we used
# for the test distributions
constants.kaon_br_map["test1"] = 0.75
constants.kaon_br_map["test2"] = 0.25

fig, ax = plt.subplots(1,3, figsize=(21,5))
histo.hist_data(data, bins=100, range=(0,10), ax=ax[0])
histo.stack_mc_flux({"test1": mc_same, "test2": mc2}, {"test1": n_simu, "test2": n_simu2}, bins=100, range=(0,10), kaon_flux=flux, ax=ax[0])
ax[0].set_title("MC generated according to same distribution as data")
ax[0].set_xlabel("x")

histo.hist_data(data, bins=100, range=(0,10), ax=ax[1])
histo.stack_mc_flux({"test1": mc_same_with_cut, "test2": mc2_with_cut}, {"test1": n_simu, "test2": n_simu2}, bins=100, range=(0,10), kaon_flux=flux, ax=ax[1])
ax[1].set_title("MC generated according to same distribution as data\n, wihth cut x>1")
ax[1].set_xlabel("x")

histo.hist_data(data, bins=100, range=(0,10), ax=ax[2])
histo.stack_mc_flux({"test1": mc, "test2": mc2}, {"test1": n_simu, "test2": n_simu2}, bins=100, range=(0,10), kaon_flux=flux, ax=ax[2])
ax[2].set_title("MC generated according to distribution where\n the amplitude of the second sigma is different")
ax[2].set_xlabel("x")

All the plots produced using this second method look better:
 - The first plot is obviously OK as the MC model correspond to the data model
 - The second plot also look good. The results produced by this normalization method does not change if the cuts applied to data and MC are not exaclty the same. The parts where they are will look good (x>1)
 - Aside from the second Gaussian, which is different by design, the rest of the distribution is correctly normalized. In particular the first gaussian still looks OK, as it should because its parameters are the same as in data.

On the other hand we needed a bit more information to produce these plots: the number of simulated MC samples and the BR (as for the previous method), but also the kaon flux. For this example, this number was just given as input (because this is a made up example), but generally this number is unknown and depends on the data sample. A complex data seletcion must be performed to estimate it, including the estimation of the uncertainty on this number. 

Any serious data analysis will however have to ultimately use this second technique, while the first one can be useful to produce some quick, preliminary plots. The obtention of the kaon flux number will be the topic of one of the next section of this notebook.

## Selections

In [ ]:
# Load the data, but also the MC simulations we have for each channel
# N.B. Contrary to what we were doing in previous notebooks, we are now
#      retrieving the second output value from import_root_files.
data, data_norm = prepare.import_root_files(["data/run12450.root"])
k2pi, k2pi_norm = prepare.import_root_files(["data/k2pi.root"])
k3pi, k3pi_norm = prepare.import_root_files(["data/k3pi.root"])
kmu2, kmu2_norm = prepare.import_root_files(["data/kmu2.root"])
kmu3, kmu3_norm = prepare.import_root_files(["data/kmu3.root"])
ke3, ke3_norm = prepare.import_root_files(["data/ke3.root"])

# And we create a dictionary with the normalization parameters that we retrieved
normalization_dict = {"k2pi": k2pi_norm, "k3pi": k3pi_norm, "kmu2": kmu2_norm, "kmu3": kmu3_norm, "ke3": ke3_norm}

In [ ]:
# Compute the weights that we will need for plotting
total_data = len(data)
weights = histo.compute_samples_weights(normalization_dict)

### One track selection
As you can see, we have several channels (all except K3pi) which feature only one single track in the final state. It therefore makes sense to try to have a common selection allowing us to select those channels in a similar way. Later on this has the advantage that the comparison between samples from each channel will have similar systematic errors, because the selection criterion are as similar as possible.

*Note*: All examples in this section will use the first method of normalization (scaling to data) because we do not have a normalization selection yet.

In [ ]:
# Let's first create a function that will plot the invariant mass for the data and MC passed, this will save us a lot of code later
from typing import Dict, Tuple, List, Callable
def plot_invariant_mass(data: pd.DataFrame, mc_dict: Dict[str, pd.DataFrame], weights_dict: Dict[str, float], 
                        mass_assignment: Dict) -> Tuple[int, int]:
    # Plot the histogram for data
    ndata = histo.hist_data(hlf.invariant_mass(data, mass_assignment), bins=400, range=(200,600))

    # Plot the histogram stack for the MC samples
    nmc = histo.stack_mc_scale([hlf.invariant_mass(mc_dict[mc_name], mass_assignment) for mc_name in mc_dict.keys()], 
        bins=400, range=(200, 600), weights=weights_dict, labels=mc_dict.keys(), ndata=ndata)

    # Some default display parameters
    plt.legend()
    plt.yscale("log")
    plt.ylim(bottom=0.8)

    # Return the number of data events plotted and a dictionary of number of MC events plotted by sample
    return ndata, nmc

Let's start first by selecting all single-track with two-cluster events. From this we can plot the invariant mass. In the following we will work in the pion hypothesis (considering k2pi as our test case).

In [ ]:
# Make the k2pi assumption
k2pi_mass_assignment = {"track1": constants.pion_charged_mass, "cluster1": constants.photon_mass, "cluster2": constants.photon_mass}
plot_invariant_mass_k2pi = partial(plot_invariant_mass, mass_assignment=k2pi_mass_assignment)

In [ ]:
ndata, nmc = plot_invariant_mass_k2pi(data, {"k3pi": k3pi, "kmu2": kmu2, "k2pi": k2pi, "kmu3": kmu3, "ke3": ke3}, weights)
plt.title("Reconstructed $\pi^+\gamma\gamma$ invariant mass")
plt.xlabel("$m_{\pi^+\gamma\gamma}$ [MeV/$c^2$]")
print(f"K2pi selection purity: {nmc['k2pi']/sum(nmc.values()):.2%}")

The Data/MC agreement does not look good. The reason is that we use an inconsistent mix of various pre-selections with various efficiencies depending on the sample. We did not even select events with 1 Track and 2 clusters, which means we have really included in the plot random events.

Furthermore as already mentioned, the pre-selections have a set of similar cuts, but not necessarily at the same values. We will therefore reconcile these cuts to a common value as described earlier. We define hereafter the list of cuts that can be applied to all (1-track & 2 cluster) selections.

In [ ]:
# Select the correct topology
cond_1T2C = hlf.make_exists_cut(["track1", "cluster1", "cluster2"], [])

# Distance/position cuts
lkr_dtrack_cluster_cond = partial(hlf.make_lkr_distance_cut, 150, None)
lkr_dclusters_cond = hlf.make_lkr_distance_cut(200, None, "cluster1", "cluster2")
z_vertex_cond = hlf.make_z_vertex_cut(120000, 180000)
cda_cond = hlf.make_cda_cut(None, 25)
neutral_vtx_cond = hlf.make_charged_neutral_vertex_cut(None, 10000, "cluster1", "cluster2", constants.pion_neutral_mass)

# Energy/momentum cuts
cluster_energy_cond = partial(hlf.make_energy_cut, 2000, None)

# PID cuts
rich_e_cond = partial(hlf.make_rich_cut, constants.rich_hypothesis_map["e"])
rich_pi_cond = partial(hlf.make_rich_cut, constants.rich_hypothesis_map["pi"])
rich_mu_cond = partial(hlf.make_rich_cut, constants.rich_hypothesis_map["mu"])
lkr_e_cond = partial(hlf.make_eop_cut, 0.95, 1.05)
lkr_pi_cond = partial(hlf.make_eop_cut, None, 0.9)
lkr_mu_cond = partial(hlf.make_eop_cut, None, 0.2)
muv3_mu_cond = partial(hlf.make_muv3_cut, True, time_window=1.5)
muv3_not_mu_cond = partial(hlf.make_muv3_cut, False, time_window=2.5)

Then we restart the plot selecting at least the correct topology

In [ ]:
data_sel, mc_sel = hlf.select_all(data, {"k3pi": k3pi, "kmu2": kmu2, "k2pi": k2pi, "kmu3": kmu3, "ke3": ke3}, [cond_1T2C])
ndata, nmc = plot_invariant_mass_k2pi(data_sel, mc_sel, weights)
plt.title("Reconstructed $\pi^+\gamma\gamma$ invariant mass")
plt.xlabel("$m_{\pi^+\gamma\gamma}$ [MeV/$c^2$]")
print(f"K2pi selection purity: {nmc['k2pi']/sum(nmc.values()):.2%}")

This is already better, but we still have issues. Let's apply the remaining cuts to align the selections.

In [ ]:
data_sel, mc_sel = hlf.select_all(data_sel, mc_sel, 
                                  [z_vertex_cond, lkr_dtrack_cluster_cond("track1", "cluster1"), lkr_dtrack_cluster_cond("track1", "cluster2"),
                                   lkr_dclusters_cond, cluster_energy_cond("cluster1"), cluster_energy_cond("cluster2"), 
                                   cda_cond, neutral_vtx_cond])
ndata, nmc = plot_invariant_mass_k2pi(data_sel, mc_sel, weights)
plt.title("Reconstructed $\pi^+\gamma\gamma$ invariant mass")
plt.xlabel("$m_{\pi^+\gamma\gamma}$ [MeV/$c^2$]")
print(f"K2pi selection purity: {nmc['k2pi']/sum(nmc.values()):.2%}")

These cuts have not changed the picture much. We can confirm this by looking at the acceptance of each cut (i.e. which additional fraction of events the cut will let pass).

In [ ]:
mc_sel["k2pi"].attrs["acceptances"]

We can see that indeed each cut has less than 1% effect. The reason however is clear: these cuts have already been applied in the pre-selection, we are just applying fine tuning here (like cutting at cda<25 instead of cda<30).

There are still two major differences between the selections: the PID and the reconstructed kinematic cuts. At this point however we cannot remain generic and we have to actually select specific channels. We will embrace full on the k2pi selection and move to other selections later.

In [ ]:
# Define K2pi reconstructed kinematic cuts
k2pi_mmiss2_cond = hlf.make_missing_mass_sqr_cut(min_mm2=None, max_mm2=0.015*1e6, mass_assignments=k2pi_mass_assignment) # In GeV
k2pi_inv_mass_cond = hlf.make_invariant_mass_cut(min_mass=460, max_mass=520, mass_assignments=k2pi_mass_assignment)

In [ ]:
# Apply all PID techniques
data_sel_pid, mc_sel_pid = hlf.select_all(data_sel, mc_sel, [lkr_pi_cond(which_track="track1"), 
                                                             muv3_not_mu_cond(which_track="track1"), 
                                                             rich_pi_cond(which_track="track1")])
ndata, nmc = plot_invariant_mass_k2pi(data_sel_pid, mc_sel_pid, weights)
plt.ylim(bottom=1e-2)
plt.title("Reconstructed $\pi^+\gamma\gamma$ invariant mass")
plt.xlabel("$m_{\pi^+\gamma\gamma}$ [MeV/$c^2$]")
print(f"K2pi selection pollution (1-purity): {1-nmc['k2pi']/sum(nmc.values()):.2e}")
nmc

Much better, but we can still see a total of ~0.84 estimated background events mostly due to k3pi (for a pollution at the order of $4.5\times10^{-5}$ which is already very good). We can further improve by adding the final kinematic cuts.

In [ ]:
data_sel_pid, mc_sel_pid = hlf.select_all(data_sel_pid, mc_sel_pid, [k2pi_mmiss2_cond, k2pi_inv_mass_cond])
ndata, nmc = plot_invariant_mass_k2pi(data_sel_pid, mc_sel_pid, weights)
plt.ylim(bottom=1e-2)
plt.title("Reconstructed $\pi^+\gamma\gamma$ invariant mass")
plt.xlabel("$m_{\pi^+\gamma\gamma}$ [MeV/$c^2$]")
print(f"K2pi selection pollution (1-purity): {1-nmc['k2pi']/sum(nmc.values()):.2e}")
nmc

This last operation gained us another order of magnitude in purity. But what about the acceptance?

In [ ]:
display(mc_sel_pid["k2pi"].attrs["acceptances"])
print(f"K2pi selection acceptance: {len(mc_sel_pid['k2pi'])/normalization_dict['k2pi']:.2%}")

We see here that the total selection acceptance is 9.08%, which means that out of all K2pi events simulated, only 9.08% are making it to the end of the selection. This seems not a lot but that is in fact the typical perforformances of a selection in NA62 if we want a pure sample. The acceptance of each individual cut that we have applied is high, and as already mentioned the reason is that most of them have already been applied in the pre-selection and their full performance is actually included in the original 15.7% acceptance of the pre-selection. 

What stands out is that two cuts in particular have a large impact: 
 - The cut on the z vertex. In fact moving from 105 m to 120 m reduces the allowed decay volume by 20%. This translates to a 15% loss in acceptance (because of the kinematics and geometrical acceptance, the vertex Z distribution is not completely uniform - see next plot). This is a large loss, but unfortuntely a necessary one if we want to keep this selection as close as possible to the other 1-track & 2 clusters selections.
 - The RICH cut on the PID induces an enormous acceptance loss of 31.5% (relative). Let's see if if it is really necessary or if E/P and MUV3 is enough?

In [ ]:
# Plot the Z vertex distribution of k2pi to illustrate the point made in the previous paragraph.
k2pi["vtx_z"].hist(bins=80, range=(100000, 180000))
plt.title("K2pi sample vertex Z distribution")
plt.xlabel("$Z_\mathrm{vtx}$ [mm]")

In [ ]:
# Apply PID techniques but not RICH, directly apply also the reconstructed kinematics cuts
data_sel_pid, mc_sel_pid = hlf.select_all(data_sel, mc_sel, [lkr_pi_cond(which_track="track1"), muv3_not_mu_cond(which_track="track1"), 
                                                             k2pi_mmiss2_cond, k2pi_inv_mass_cond])
ndata, nmc = plot_invariant_mass_k2pi(data_sel_pid, mc_sel_pid, weights)
plt.ylim(bottom=1e-2)
plt.title("Reconstructed $\pi^+\gamma\gamma$ invariant mass")
plt.xlabel("$m_{\pi^+\gamma\gamma}$ [MeV/$c^2$]")
print(f"K2pi selection pollution (1-purity): {1-nmc['k2pi']/sum(nmc.values()):.2e}")
nmc

In [ ]:
display(mc_sel_pid["k2pi"].attrs["acceptances"])
print(f"K2pi selection acceptance: {len(mc_sel_pid['k2pi'])/normalization_dict['k2pi']:.2%}")

The selection acceptance has increased by 4% absolute. But on the other hand the purity has decreased by two orders of magnitude! Depending on our objective (purity vs. acceptance) we can decide to use this extra PID condition or not.

### Normalization selection

Let's assume in the following that we are interested in a pure selection, because we do not actually want to measure K2pi but only use it as normalization. In NA62 this process is necessary as we do not have an absolute measurement of the kaon flux (i.e. how many kaons enter our experiment). Instead we must use a well known normalization channel (such as K2pi), measure a flux based on a normalization selection (a pure selection for K2pi) and measure our signal channel relative to the normalization channel. We can then move from a relative value to an absolute value by using the K2pi BR as an external input (with its own uncertainties, which is why we want to use a channel with existing precision measurements).

As an example, the process for measuring the Ke3 branching ratio using K2pi as normalization is the following:
 - Perform the K2pi and Ke3 selections, estimate the K2pi selection acceptance $A_\text{k2pi}$ (on K2pi events) and Ke3 selection acceptance $A_\text{ke3}$ (on Ke3 events).
 - We know that in general the number of selected events is $N_\text{sel} = N_K *\text{BR} * A * \varepsilon_\text{trigg}$ where $N_K$ is the kaon flux, BR is the branching fraction of the channel, $A$ is the selection acceptance, and $\varepsilon_\text{trigg}$ is the trigger efficiency.
 - Applying this to the K2pi normalization we have: $$N_K = \frac{N_\text{k2pi,sel}}{A_\text{k2pi}\cdot\text{BR(k2pi)}\cdot\varepsilon_\text{trigg,k2pi}}$$
 - We have the equivalent equation for Ke3, into which we can substitute $N_K$ estimated from K2pi: $$N_\text{ke3,sel} = N_K\cdot A_\text{ke3}\cdot\text{BR(ke3)}\cdot\varepsilon_{trigg,ke3}$$
$$\Rightarrow\text{BR(ke3)} = \frac{N_\text{ke3,sel}}{N_K\cdot A_\text{ke3}\cdot\varepsilon_{trigg,ke3}}$$
$$\Rightarrow\text{BR(ke3)} = \frac{N_\text{ke3,sel}}{N_\text{k2pi,sel}}\cdot\frac{A_\text{k2pi}}{A_\text{ke3}}\cdot\frac{\varepsilon_\text{trigg,k2pi}}{\varepsilon_{trigg,ke3}}\cdot\text{BR(k2pi)}$$
This last expression gives us our final measurement as a function of the external parameter BR(k2pi). As you can see we also take ratios of equivalent quantities between Ke3 and K2pi. This means that in the assumption that this quantities are obtained and apply in a similar way for both channels (selections are closely related, trigger lines are the same and applying on similar conditions, data are acquired at the same time, ...) then they will have similar systematic uncertainties and they will cancel out in the ratio at first order.

In the following we will also assume that the trigger efficiencies are 100% (real value is very high anyways, well above 90%). Please note also that the Kaon flux computed is an **effective** flux, not the absolute one. It depends on the trigger line, on the trigger downscaling, on some geometry parameters, ...

In [ ]:
# Let's define our complete K2pi selection
common_1track_2cluster_selection = [cond_1T2C, z_vertex_cond, 
                                    lkr_dtrack_cluster_cond("track1", "cluster1"), lkr_dtrack_cluster_cond("track1", "cluster2"), lkr_dclusters_cond, 
                                    cluster_energy_cond("cluster1"), cluster_energy_cond("cluster2"), 
                                    cda_cond, neutral_vtx_cond]

k2pi_selection = [lkr_pi_cond(which_track="track1"), muv3_not_mu_cond(which_track="track1"), rich_pi_cond(which_track="track1"), 
                  k2pi_mmiss2_cond, k2pi_inv_mass_cond]

Having defined our K2pi selection for normalization, we can compute the flux $N_\text{k2pi,sel}$

In [ ]:
# Select first common 1-track & 2-clusters sample so we can use it multiple times without re-selecting it
data_1T2C, mc_1T2C = hlf.select_all(data, {"k3pi": k3pi, "kmu2": kmu2, "k2pi": k2pi, "kmu3": kmu3, "ke3": ke3}, common_1track_2cluster_selection)

In [ ]:
# Perform the k2pi selection
data_k2pi, mc_k2pi = hlf.select_all(data_1T2C, mc_1T2C, k2pi_selection)

In [ ]:
n_k2pi_mc_sel = len(mc_k2pi["k2pi"])
n_k2pi_data_sel = len(data_k2pi)
k2pi_acc = n_k2pi_mc_sel/normalization_dict["k2pi"]
k2pi_N_K = n_k2pi_data_sel/(k2pi_acc * constants.kaon_br_map["k2pi"] * 1.0)

print(f"K2pi selection acceptance: {k2pi_acc:.2%}")
print(f"Number of K2pi candidates in data: {len(data_k2pi)} candidates")
print(f"Kaon flux: {k2pi_N_K:.2e} kaons")

### Signal selection
We are now interested in selecting the signal. For this exercise we are going to use Ke3 as the signal. As mentionned earlier we want to have our signal selection as close as possible to our normalization selection to cancel out the systematics in the ratio. We are therefore going to start with the common 1-track & 2-clusters selection, and add the few PID and reconstructed kinematic cuts specific to Ke3.

Ke3 is an opened-kinematics channel, so we cannot use the reconstructed invariant mass to constrain it. But the missing mass squared must peak at 0, so we can use that one.

In [ ]:
# Define the missing mass squared plotting function as we did for the invariant mass earlier
def plot_missing_mass2(data: pd.DataFrame, mc_dict: Dict[str, pd.DataFrame], kaon_flux: float, normalization: Dict[str, float],
                        mass_assignment: Dict) -> Tuple[int, int]:
    # Plot the histogram for data
    ndata = histo.hist_data(hlf.missing_mass_sqr(data, mass_assignment), bins=100, range=(-50000, 100000))

    # Plot the histogram stack for the MC samples
    nmc = histo.stack_mc_flux({mc_name: hlf.missing_mass_sqr(mc_dict[mc_name], mass_assignment) for mc_name in mc_dict.keys()}, normalization,
        bins=100, range=(-50000, 100000), labels=mc_dict.keys(), kaon_flux=kaon_flux)

    # Some default display parameters
    plt.legend()
    plt.yscale("log")
    plt.ylim(bottom=0.8)

    # Return the number of data events plotted and a dictionary of number of MC events plotted by sample
    return ndata, nmc

# Make the Ke3 assumption
ke3_mass_assignment = {"track1": constants.electron_mass, "cluster1": constants.photon_mass, "cluster2": constants.photon_mass}
plot_missing_mass2_ke3 = partial(plot_missing_mass2, mass_assignment=ke3_mass_assignment, kaon_flux=k2pi_N_K, normalization=normalization_dict)

In [ ]:
ndata, nmc = plot_missing_mass2_ke3(data_1T2C, mc_1T2C)
plt.ylim(bottom=1e-2)
plt.title("Missing mass squared (electron hypthesis)")
plt.xlabel("$m_\mathrm{miss}^2(e)$ [$\mathrm{MeV}^{2}/c^{4}]$")
print(f"Ke3 selection pollution (1-purity): {1-nmc['ke3']/sum(nmc.values()):.2e}")
nmc

This was the result of the common 1-track & 2-clusters selection, without applying yet any Ke3 specific cuts. 

There are two reasons for applying a cut:
 - Removing a phase space region including more background than signal. This is mostly what we have been doing so far.
 - Removing a region of phase space which is not reproduced well in MC, thus removing possible sources of systematic uncertainties in your final result.

While performing an analysis, you will want to plot as many observables as possible, always comparing data and MC. Certainly you will want to check all those on which you are applying a cut. You want to make sure that the region that you keep agrees well between data and MC. Of course your initial data are polluted with many channels and effects not reproduced in MC, so you do not want to do your Data/MC comparison on the "raw" data without any selection cut. Very often, what you want to do is to apply all or most of your cuts, except the one you are checking.

Let's prepare some code to do that.

In [ ]:
from typing import Union, List, Dict, Callable
# Create a function that will apply all the cuts to the data and MC, except a cut we want to check,
# then plot the relevant distribution
def plot_before_cut(data: pd.DataFrame, mc_dict: Dict[str, pd.DataFrame], 
                    selection: List[Callable], cut: Union[Callable, str], 
                    transform: Union[Callable, str], 
                    bins: int = 100, range: Union[None, Tuple[int, int]] = None, ax: Union[None, plt.Axes] = None) -> None:
    # Cut can be either the cut function we want to remove, or the name of the cut function
    # Example: z_vertex_cond = make_z_vertex_cut(105000, 180000)
    #   - Call with cut=z_vertex_cond
    #   - Or call with cut='z_vertex_cut'
    # Transform indicates which/how the plotted value should be extracted from the dataframe
    
    truncated_selection = hlf.remove_cut(selection, cut)

    # Perform truncated selection on data and MC
    d, m = hlf.select_all(data, mc_dict, truncated_selection)

    # Two cases: transform is a string and should correspond to a column of the dataframe, or a callable to be applied to the dataframe
    if isinstance(transform, str):
        # Transform it into a callable so we do not have to distinguish later
        col_name = transform
        transform = lambda df: df[col_name]
    # And plot
    histo.hist_data(transform(d), bins=bins, range=range, ax=ax)
    # N.B. We use here two variables external to the function. This will work only in the notebook after we ran the cells generating the
    # variables 'normalization_dict' and 'k2pi_N_K'
    histo.stack_mc_flux({_: transform(m[_]) for _ in m}, normalization_dict, bins=bins, range=range, kaon_flux=k2pi_N_K, ax=ax)

# Just for convenience later so we have less code to type
data_orig = data
mc_dict_orig = {"k3pi": k3pi, "kmu2": kmu2, "k2pi": k2pi, "kmu3": kmu3, "ke3": ke3}

Equipped with this, let's check the Data/MC agreement for the distributions on which we perform cuts.

In [ ]:
# Z vertex cut
plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, z_vertex_cond, "vtx_z", range=(100000, 180000))
plt.xlabel("$Z_\mathrm{vtx}$ [mm]")
plt.legend()
plt.title("Vertex Z distribution")

In [ ]:
# LKr distance cut (3 different cuts: track-cl1, track-cl2, cl1-cl2)
fig, ax = plt.subplots(1, 3, figsize=(21, 5))
plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, "lkr_distance_cut", 
                partial(hlf.lkr_distance, object_1="track1", object_2="cluster1"), range=(0, 3500), ax=ax[0])
ax[0].legend()
ax[0].set_title("LKr distance: track-cluster1")
ax[0].set_xlabel("$d_\mathrm{track,cluster}$ [mm]")

plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, "lkr_distance_cut", 
                partial(hlf.lkr_distance, object_1="track1", object_2="cluster2"), range=(0, 3500), ax=ax[1])
ax[1].legend()
ax[1].set_title("LKr distance: track-cluster2")
ax[1].set_xlabel("$d_\mathrm{track,cluster}$ [mm]")

plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, "lkr_distance_cut", 
                partial(hlf.lkr_distance, object_1="cluster1", object_2="cluster2"), range=(0, 2000), ax=ax[2])
ax[2].legend()
ax[2].set_title("LKr distance: cluster1-cluster2")
ax[2].set_xlabel("$d_\mathrm{cluster,cluster}$ [mm]")

In [ ]:
# Cluster energy cuts
fig, ax = plt.subplots(1, 2, figsize=(14, 5))
plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, "energy_cut", "cluster1_lkr_energy", range=(0, 60000), ax=ax[0])
ax[0].legend()
ax[0].set_title("Cluster energy")
ax[0].set_xlabel("E [MeV]")

plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, "energy_cut", "cluster2_lkr_energy", range=(0, 60000), ax=ax[1])
ax[1].legend()
ax[1].set_title("Cluster energy")
ax[1].set_xlabel("E [MeV]")

In [ ]:
# CDA cut
plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, cda_cond, hlf.beam_vertex_cda, range=(0,20))
plt.xlabel("CDA [mm]")
plt.legend()
plt.title("CDA distribution")

In [ ]:
# CDA cut
plot_before_cut(data_orig, mc_dict_orig, common_1track_2cluster_selection, neutral_vtx_cond, 
                partial(hlf.charged_neutral_distance, cluster_1="cluster1", cluster_2="cluster2", clusters_invariant_mass=constants.pion_neutral_mass), range=(0,250000))
plt.xlabel("$d_\mathrm{neutral,charged}$ [mm]")
plt.legend()
plt.title("Distance between charged and neutral vertex")
plt.yscale("log")

We have gone here through all the common 1-track & 2-clusters cuts. The agreement between Data/MC looks very reasonable in all of the variables, with the possible exception of the CDA. We will keep this in mind for later and look for possible systematic effects due to this cut.

Let's complete our selection with PID and reconstructed kinematics.

In [ ]:
# Let's define our specifc Ke3 selection
ke3_mmiss2_cond = hlf.make_missing_mass_sqr_cut(min_mm2=None, max_mm2=0.01*1e6, mass_assignments=ke3_mass_assignment) # In GeV
ptot_neutrino_cond = hlf.make_total_momentum_cut(15000, 70000)
pt_neutrino_cond = hlf.make_transverse_momentum_cut(40, 250)
  
ke3_selection = [lkr_e_cond(which_track="track1"), # No other PID conditions necessary here, E/p is very good for electrons
                 ptot_neutrino_cond, pt_neutrino_cond, ke3_mmiss2_cond]

In [ ]:
data_ke3, mc_ke3 = hlf.select_all(data_1T2C, mc_1T2C, ke3_selection)
ndata, nmc = plot_missing_mass2_ke3(data_ke3, mc_ke3)
plt.ylim(bottom=1e-2)
plt.title("Missing mass squared (electron hypthesis)")
plt.xlabel("$m_\mathrm{miss}^2(e)$ [$\mathrm{MeV}^{2}/c^{4}]$")
print(f"Ke3 selection pollution (1-purity): {1-nmc['ke3']/sum(nmc.values()):.2e}")
nmc

In [ ]:
display(mc_ke3["ke3"].attrs["acceptances"])
print(f"Ke3 selection acceptance: {len(mc_ke3['ke3'])/normalization_dict['ke3']:.2%}")

This gives us an acceptance of 7.4%, not too bad, for an excellent purity at the level of $6\times10^{-5}$.

Before continuing, let's check also the Ke3 Data/MC agreement in the distributions of the cuts specific to Ke3

In [ ]:
# E/P cut
plot_before_cut(data_1T2C, mc_1T2C, ke3_selection, "eop_cut", "track1_eop", range=(0,1.5))
plt.xlabel("$E/p$")
plt.legend()
plt.title("Track E/p")
plt.yscale("log")

In [ ]:
# Total momentum cut
plot_before_cut(data_1T2C, mc_1T2C, ke3_selection, ptot_neutrino_cond, hlf.total_momentum, range=(0,80000))
plt.xlabel("$p_\mathrm{tot}$ [MeV/c]")
plt.legend()
plt.title("Total reconstructed momentum")

In [ ]:
# Transverse momentum cut
plot_before_cut(data_1T2C, mc_1T2C, ke3_selection, pt_neutrino_cond, hlf.transverse_momentum_beam, range=(0,250))
plt.xlabel("$p_\mathrm{t}$ [MeV/c]")
plt.legend()
plt.title("Total transverse momentum with respect to Beam momentum")

In [ ]:
# Missing mass squared cut
plot_before_cut(data_1T2C, mc_1T2C, ke3_selection, ke3_mmiss2_cond, partial(hlf.missing_mass_sqr, momenta_or_masses=ke3_mass_assignment), range=(-20000,20000))
plt.xlabel("$m_\mathrm{miss}^2(e)$ [$\mathrm{MeV}^2/c^{4}$]")
plt.legend()
plt.title("Missing mass squared (electron hypothesis)")

Again everything looks good, except maybe for some small disagreement in the E/p variable, to be kept in mind for later. Note also the sharp cut on the total reconstructed momentum, even not applying our own cut. The reason is that the cut has already been applied in the pre-selection, and we are therefore unable to check the whole range in this exercise.

## Measurement
### Ke3 branching ratio measurement
If we want now to perform the measurement of BR(Ke3) (normalized on K2pi), we can use the formula mentioned earlier and run the computation. As a reminder, we add it again here:
$$\text{BR(ke3)} = \frac{N_\text{ke3,sel}}{N_\text{k2pi,sel}}\cdot\frac{A_\text{k2pi}}{A_\text{ke3}}\cdot\frac{\varepsilon_\text{trigg,k2pi}}{\varepsilon_{trigg,ke3}}\cdot\text{BR(k2pi)}$$

In [ ]:
br_k2pi = constants.kaon_br_map["k2pi"]
epsilon_trigg_ratio = 1./1. # We decided to consider the trigger efficiency to be 100%
ke3_acc = len(mc_ke3["ke3"])/normalization_dict["ke3"]
acc_ratio = k2pi_acc / ke3_acc
sel_ratio = len(data_ke3)/len(data_k2pi)
br_ke3_meas = sel_ratio * acc_ratio * epsilon_trigg_ratio * br_k2pi
print(f"Measured Ke3 branching ratio: {br_ke3_meas:.2%}")
print(f"Most precise Ke3 branching ratio (PDG): {constants.kaon_br_map['ke3']:.2%}")

Congratulations! You have now measured the Ke3 branching fraction, and obtained a value that is very close to the best measurement. We have however not provided any uncertainties on that value yet. So this could be compatible by chance if we have very large uncertainties (e.g. of the order of 10%), or we could have a very precise measurement instead, still compatible if the uncertainties are of the order of 0.1%, or incompatible if our uncertainties are even smaller (e.g. 0.01%). 

### Uncertainties estimates
In summary, we cannot conclude anything if we do not measure the uncertainties.

We could of course perform ourselves the task of computing all the uncertainties and propagating them to the end result manually. In python there is however a nice package called "uncertainties" which performs this in an automatic way. Although we have to be careful during complex operations as there may be additional subtelties not taken into account by the package. But we are here going to perform simple multiplication/division/addition operations without on simple numbers associated with uncertainties, and not correlated (although they are also taken care of automatically).

So we can repeat all our previous operations, but adding uncertainties to our numbers.

In [ ]:
from uncertainties import ufloat
# Let's make it a function as we will want to compute these numbers several more times later
def compute_normalization(selected_data, selected_mc, br, mc_normalization):
    n_mc_sel = selected_mc if isinstance(selected_mc, int) else len(selected_mc)
    n_data_sel = selected_data if isinstance(selected_data, int) else len(selected_data)
    
    # Add poisson uncertainties 
    n_mc_sel = ufloat(n_mc_sel, np.sqrt(n_mc_sel), "MC K2pi")
    n_data_sel = ufloat(n_data_sel, np.sqrt(n_data_sel), "Data K2pi")
    
    # No uncertainty on the normalization: we know exactly how many MC events were generated for this sample
    acc = n_mc_sel/mc_normalization
    n_K = n_data_sel/(acc * br * 1.0)

    return acc, n_data_sel, n_K

br_k2pi = ufloat(constants.kaon_br_map["k2pi"], 0.0008, "K2pi BR")
k2pi_acc, n_k2pi_data_sel, k2pi_N_K = compute_normalization(data_k2pi, mc_k2pi["k2pi"], br_k2pi, normalization_dict["k2pi"])
print(f"K2pi selection acceptance: {k2pi_acc:%}")
print(f"Number of K2pi candidates in data: {n_k2pi_data_sel} candidates")
print(f"Kaon flux: {k2pi_N_K:e} kaons")

In [ ]:
# Let's also make it a function as we will also repeat it a few times

def compute_signal_br(selected_data, selected_mc, mc_normalization, flux):
    n_mc_sel = selected_mc if isinstance(selected_mc, int) else len(selected_mc)
    n_data_sel = selected_data if isinstance(selected_data, int) else len(selected_data)

    # Add poisson uncertainties
    n_mc_sel = ufloat(n_mc_sel, np.sqrt(n_mc_sel), "MC Ke3")
    n_data_sel = ufloat(n_data_sel, np.sqrt(n_data_sel), "Data Ke3")

    # Rather than computing each ratio separately, we use directly the flux which include all 
    # the relevant K2pi values (so we don't have to pass all the values individually to the function)
    acc = n_mc_sel/mc_normalization
    br_ke3_meas = n_data_sel / (acc * 1. * flux)

    return acc, n_data_sel, br_ke3_meas

ke3_acc, n_ke3_data_sel, br_ke3_meas = compute_signal_br(data_ke3, mc_ke3["ke3"], normalization_dict["ke3"], k2pi_N_K)
br_ke3 = ufloat(constants.kaon_br_map['ke3'], 0.0004)
print(f"Measured Ke3 branching ratio: {br_ke3_meas:%}")
print(f"Most precise Ke3 branching ratio (PDG): {br_ke3:%}")

We can now conclude: our measurement is slightly less precise that the most precise found in the PDG, but not by much (0.09% uncertainties vs. 0.04%), and they are perfectly compatible. However we have taken into account only statistical uncertainties, not any systematic uncertainties. As previously mentioned, we expect them to be small as they will cancel out in the ratiom but still they should be taken into account and could possibly be of a similar magnitude to the statistical uncertainties.

### Systematic uncertainties
Systematic uncertainties are linked to possible bias that could be introduced during the selection/analysis process. 
While checking the data/MC agreement in the distribution on which we introduce cuts in the previous section, two of them (the CDA and the electron E/p) had some level of disagreement and we mentioned that we should keep those in mind. It is now the time to see if those would introduce possible bias on the final result. The possible bias that could be introduced will depend on the threshold we choose for the cut and will result in the final measurement changing (slightly or more) depending on which value we choose. The way to check for systematic bias and assign uncertainties is therefore to perform the selection while varying the cut value around the nominal one.

Technically we will achieve this by removing the original cut from the selection, and repeat the whole selection-measurement process introducing the cut with a new value.

In [ ]:
from tqdm.notebook import tqdm
# Let's start with a few useful functions

# This function will run the process described above:
#  - Take as input the normalization and signal selections
#  - Which cut to remove and what replacements we will use (and if the change must happen on both normalization and signal selections, or only one of them
#  - Some other variables that are needed by the compute_normalization and compute_signal_br functions
# Then it will run over all the alternative cuts, recording all the useful final numbers in an array, for each tested alternative
def systematic_checks(normalization_selection: List[Callable], signal_selection: List[Callable], 
                      test_cut: Union[Callable, str], replacement_cuts: List[Callable], 
                      br_norm: float, norm_name: str, signal_name: str, truncate_which: str = "both") -> Tuple[float]:

    # Truncate the selections if necessary
    truncated_norm_selection = hlf.remove_cut(normalization_selection, test_cut) if truncate_which in ["both", "norm"] else normalization_selection
    truncated_signal_selection = hlf.remove_cut(signal_selection, test_cut) if truncate_which in ["both", "signal"] else signal_selectionl

    # Prepare lists for output
    flux_list = []
    n_norm_list = []
    acc_norm_list = []
    n_sig_list = []
    acc_sig_list = []
    br_sig_list = []

    # Loop over the alternative cuts
    for new_cond in tqdm(replacement_cuts):
        # Perform the alternative selection
        norm_selection = truncated_norm_selection + [new_cond] if truncate_which in ["both", "norm"] else truncated_norm_selection
        signal_selection = truncated_signal_selection + [new_cond] if truncate_which in ["both", "signal"] else truncated_signal_selection
        data_test_norm, mc_test_norm = hlf.select_all(data_orig, mc_dict_orig, norm_selection)
        data_test_signal, mc_test_signal = hlf.select_all(data_orig, mc_dict_orig, signal_selection)

        # Compute the final numbers
        norm_acc, n_norm_data_sel, flux = compute_normalization(data_test_norm, mc_test_norm[norm_name], br_norm, normalization_dict[norm_name])
        sig_acc, n_sig_data_sel, br_sig_meas = compute_signal_br(data_test_signal, mc_test_signal[signal_name], normalization_dict[signal_name], flux)

        # Populate the output lists
        flux_list.append(flux)
        n_norm_list.append(n_norm_data_sel)
        acc_norm_list.append(norm_acc)
        n_sig_list.append(n_sig_data_sel)
        acc_sig_list.append(sig_acc)
        br_sig_list.append(br_sig_meas)

    return flux_list, n_norm_list, acc_norm_list, n_sig_list, acc_sig_list, br_sig_list

Now perform the systematic checks (each will take a bit of time as we have to run the whole selections (4) N times).

First for the CDA. The nominal value was 25, so we can test from 15 to 40 in steps of 5.

In [ ]:
cda_values = list(np.arange(15, 40+5, 5))
replacement_cuts = [hlf.make_cda_cut(None, cda_value) for cda_value in cda_values]
flux_list, n_k2pi_list, acc_k2pi_list, n_ke3_list, acc_ke3_list, br_ke3_list = systematic_checks(common_1track_2cluster_selection + k2pi_selection, 
                                                                                                  common_1track_2cluster_selection + ke3_selection, 
                                                                                                  cda_cond, replacement_cuts, 
                                                                                                  br_k2pi, "k2pi", "ke3")

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(21, 10))
histo.plot_variations(cda_values, flux_list, 2, "$N_{K,\mathrm{k2pi}}$", "CDA", ax[0][0])
histo.plot_variations(cda_values, n_k2pi_list, 2, "$N_\mathrm{k2pi}$", "CDA", ax[0][1])
histo.plot_variations(cda_values, acc_k2pi_list, 2, "$A_\mathrm{k2pi}$", "CDA", ax[0][2])

histo.plot_variations(cda_values, n_ke3_list, 2, "$N_\mathrm{ke3}$", "CDA", ax[1][0])
histo.plot_variations(cda_values, acc_ke3_list, 2, "$A_\mathrm{ke3}$", "CDA", ax[1][1])
histo.plot_variations(cda_values, br_ke3_list, 2, "BR(ke3)$", "CDA", ax[1][2])

We see here that varying the cut has essentially no influence at all on the final result. The changes are very small, much smaller than the statistical uncertainties.

Now for the cut on the E/p for the electron PID. This cut applies only to the signal selection (for the Ke3 electron) but is not present for the normalization K2pi selection (we do have a E/p cut for the pion PID, but it is a different cut and we did not observe and data/MC discrepancy in that distribution). The E/p cut has two values, lower and upper cuts. We will perform the variation on one edge at a time. First the lower edge where the nominal value was 0.95, so we will vary it from 0.9 to 0.97 in steps of 0.01.

In [ ]:
eop_values = list(np.arange(0.90, 0.97+0.01, 0.01))
replacement_cuts = [hlf.make_eop_cut(eop_value, 1.05, which_track="track1") for eop_value in eop_values]
flux_list, n_k2pi_list, acc_k2pi_list, n_ke3_list, acc_ke3_list, br_ke3_list = systematic_checks(common_1track_2cluster_selection + k2pi_selection, 
                                                                                                  common_1track_2cluster_selection + ke3_selection, 
                                                                                                  ke3_selection[0], replacement_cuts, br_k2pi, "k2pi", "ke3",
                                                                                                  truncate_which="signal")

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(21, 10))

histo.plot_variations(eop_values, flux_list, 5, "$N_{K,\mathrm{k2pi}}$", "Electron E/p lower cut", ax[0][0])
histo.plot_variations(eop_values, n_k2pi_list, 5, "$N_\mathrm{k2pi}$", "Electron E/p lower cut", ax[0][1])
histo.plot_variations(eop_values, acc_k2pi_list, 5, "$A_\mathrm{k2pi}$", "Electron E/p lower cut", ax[0][2])

histo.plot_variations(eop_values, n_ke3_list, 5, "$N_\mathrm{ke3}$", "Electron E/p lower cut", ax[1][0])
histo.plot_variations(eop_values, acc_ke3_list, 5, "$A_\mathrm{ke3}$", "Electron E/p lower cut", ax[1][1])
histo.plot_variations(eop_values, br_ke3_list, 5, "BR(ke3)$", "Electron E/p lower cut", ax[1][2])

As expected we see no variation in the normalization histogram (as the cut is not present in the normalization selection). Contrary to the previous check, we do see a variation in the signal measurements. In particular the BR(ke3) is changing significantly once the cut value goes closer to 1. This is expected as we are starting to cut quite hard into the actual electron sample above 0.95, and we know that such high cuts are not reasonable. Setting this point aside, we see that even though we see variations, they remain within the statistical uncertainties.

Then for the upper edge with a nominal value of 1.05, we will vary from 1.03 to 1.10 in steps of 0.01.

In [ ]:
eop_values = list(np.arange(1.03, 1.10+0.01, 0.01))
replacement_cuts = [hlf.make_eop_cut(0.95, eop_value, which_track="track1") for eop_value in eop_values]
flux_list, n_k2pi_list, acc_k2pi_list, n_ke3_list, acc_ke3_list, br_ke3_list = systematic_checks(common_1track_2cluster_selection + k2pi_selection, 
                                                                                                  common_1track_2cluster_selection + ke3_selection, 
                                                                                                  ke3_selection[0], replacement_cuts, br_k2pi, "k2pi", "ke3",
                                                                                                  truncate_which="signal")

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(21, 10))

histo.plot_variations(eop_values, flux_list, 2, "$N_{K,\mathrm{k2pi}}$", "Electron E/p upper cut", ax[0][0])
histo.plot_variations(eop_values, n_k2pi_list, 2, "$N_\mathrm{k2pi}$", "Electron E/p upper cut", ax[0][1])
histo.plot_variations(eop_values, acc_k2pi_list, 2, "$A_\mathrm{k2pi}$", "Electron E/p upper cut", ax[0][2])

histo.plot_variations(eop_values, n_ke3_list, 2, "$N_\mathrm{ke3}$", "Electron E/p upper cut", ax[1][0])
histo.plot_variations(eop_values, acc_ke3_list, 2, "$A_\mathrm{ke3}$", "Electron E/p upper cut", ax[1][1])
histo.plot_variations(eop_values, br_ke3_list, 2, "BR(ke3)$", "Electron E/p upper cut", ax[1][2])

Similarly to the lower cut, we see variations in the upper cut on the signal values. We can also do a similar assessment as previously concerning the point at 1.03, which is too close to E/p=1.0. However in this case we also see variations that are larger than the statistical uncertainties when moving away from 1, and seem to stabilize when going far enough. We can assigned a systematic uncertainty of the size of the largest variation (again excluding those that we know are unreasonable).

In [ ]:
# Ignore the first entry corresponding to the too tight cut
syst_br_list = br_ke3_list[1:]
# Compute the difference with the nominal value
diff = np.array(syst_br_list) - br_ke3_meas
# Take the largest difference
syst_error = max(diff).n

br_ke3_meas_wsyst = br_ke3_meas + ufloat(0, syst_error, "Systematic")
print(f"Systematic uncertainty assigned to the E/p cut: {syst_error:.0e}")
print(f"Final measurement with systematic uncertainties: {br_ke3_meas_wsyst:%}")
print(f"Most precise Ke3 branching ratio (PDG): {br_ke3:%}")

In [ ]:
# Display the various contributions to the uncertainties
br_ke3_meas_wsyst.error_components()

In [ ]:
# Print them in order of importance, with a more convenient formatting
print("\n".join([f"{uname.tag:<10}: {uvalue:.0e}" for uname, uvalue in sorted(br_ke3_meas_wsyst.error_components().items(), key=lambda item: item[1], reverse=True)]))

We have added a small systematic uncertainty, but our measurement is still quite precise. Congratulations!

Our measurement being dominated by statistical uncertainties from the data, we only need to increase the statistics to improve it. However, note that the systematic uncertainty is the second largest at $5\times10^{-4}$, largers than the total uncertainty on the best measurement available in the PDG. This means that if we want to improve the measurement to a level better than the PDG, we would have also to reduce the systematic uncertainties, not only the statistical ones.